# CSE 881 Project
**Group Names: Edmond Anderson, Sarah Bradford, Lacey Hamilton**

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
from scipy import sparse
from pathlib import Path

In [9]:
path = Path('data_2024')
adj_matrix = sparse.load_npz(path/'adj.npz')
feat  = np.load(path/'features.npy')
labels = np.load(path/'labels.npy')
splits = json.load(open(path/'splits.json'))
idx_train, idx_test = splits['idx_train'], splits['idx_test']

In [8]:
adj.shape

(2480, 2480)

In [2]:
adj = np.load('/Users/sarahbradford/Downloads/data_2024/adj.npz')
adj_matrix = sparse.csr_matrix((adj['data'], adj['indices'], adj['indptr']), shape=adj['shape'])
feat = np.load('/Users/sarahbradford/Downloads/data_2024/features.npy')
labels = np.load('/Users/sarahbradford/Downloads/data_2024/labels.npy')
with open('/Users/sarahbradford/Downloads/data_2024/splits.json') as f:
    splits = json.load(f)

In [6]:
adj_matrix = sparse.csr_matrix((adj['data'], adj['indices'], adj['indptr']), shape=adj['shape'])

IndexError: Index dimension must be 1 or 2

In [10]:
adj_matrix = torch.FloatTensor(adj_matrix.toarray())
node_features = torch.FloatTensor(feat)
labels = torch.LongTensor(labels)

In [11]:
class GraphDataset(Dataset):
    def __init__(self, adj_matrix, node_features, labels):
        self.adj_matrix = adj_matrix
        self.node_features = node_features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.adj_matrix[idx], self.node_features[idx], self.labels[idx]

In [12]:
print(splits.keys())

dict_keys(['idx_train', 'idx_test'])


In [13]:
idx_train = splits['idx_train']
idx_test = splits['idx_test']
idx_train = [idx for idx in idx_train if idx < len(labels)]
idx_test = [idx for idx in idx_test if idx < len(labels)]
# idx_train.sort()
# idx_test.sort()

In [22]:
train_dataset.node_features

tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
train_dataset = GraphDataset(adj_matrix[idx_train], node_features[idx_train], labels[idx_train])
test_dataset = GraphDataset(adj_matrix[idx_test], node_features[idx_test], labels[idx_test])
#val_dataset = GraphDataset(adj_matrix[idx_val], node_features[idx_val], labels[idx_val])

In [15]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
#val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [16]:
# defining the GAT layer
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GATLayer, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.attn_fc = nn.Linear(2 * out_features, 1)

    def forward(self, adj_matrix, node_features):
        h = self.fc(node_features)
        N = h.size()[0]

        # Compute attention scores
        attn_scores = torch.zeros(N, N)
        for i in range(N):
            for j in range(N):
                if adj_matrix[i, j] == 1:
                    attn_input = torch.cat([h[i], h[j]], dim=0)
                    attn_scores[i, j] = self.attn_fc(attn_input).squeeze()

        # Compute attention coefficients
        attn_coefficients = nn.functional.softmax(attn_scores, dim=1)

        # Compute output features
        h_prime = torch.zeros(N, h.size()[1])
        for i in range(N):
            for j in range(N):
                h_prime[i] += attn_coefficients[i, j] * h[j]

        return h_prime
# defining th GAT model
class GAT(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GAT, self).__init__()
        self.layer1 = GATLayer(in_features, hidden_features)
        self.layer2 = GATLayer(hidden_features, out_features)

    def forward(self, adj_matrix, node_features):
        h = self.layer1(adj_matrix, node_features)
        h = torch.relu(h)
        h = self.layer2(adj_matrix, h)
        return h
# GAT model
model = GAT(in_features=node_features.size()[1], hidden_features=8, out_features=7)



In [17]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# train the model
model.train()
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        adj_matrix, node_features, labels = data
        optimizer.zero_grad()
        outputs = model(adj_matrix, node_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))
# model testing
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        adj_matrix, node_features, labels = data
        outputs = model(adj_matrix, node_features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: %d %%' % (100 * correct / total))


[1] loss: 1.845
[2] loss: 1.455
[3] loss: 0.937
[4] loss: 0.534
[5] loss: 0.327
[6] loss: 0.228
[7] loss: 0.162
[8] loss: 0.129
[9] loss: 0.094
[10] loss: 0.065
[11] loss: 0.054
[12] loss: 0.047
[13] loss: 0.044
[14] loss: 0.040
[15] loss: 0.021
[16] loss: 0.017
[17] loss: 0.015
[18] loss: 0.013
[19] loss: 0.012
[20] loss: 0.012
[21] loss: 0.011
[22] loss: 0.010
[23] loss: 0.009
[24] loss: 0.009
[25] loss: 0.009
[26] loss: 0.008
[27] loss: 0.007
[28] loss: 0.006
[29] loss: 0.006
[30] loss: 0.006
[31] loss: 0.005
[32] loss: 0.005
[33] loss: 0.005
[34] loss: 0.005
[35] loss: 0.004
[36] loss: 0.004
[37] loss: 0.004
[38] loss: 0.004
[39] loss: 0.003
[40] loss: 0.003
[41] loss: 0.003
[42] loss: 0.003
[43] loss: 0.002
[44] loss: 0.002
[45] loss: 0.002
[46] loss: 0.002
[47] loss: 0.002
[48] loss: 0.002
[49] loss: 0.002
[50] loss: 0.002
Accuracy: 20 %


In [26]:
# improve the accuracy by adding more layers
class ImprovedGAT(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(ImprovedGAT, self).__init__()
        self.layer1 = GATLayer(in_features, hidden_features)
        self.layer2 = GATLayer(hidden_features, hidden_features)
        self.layer3 = GATLayer(hidden_features, out_features)

    def forward(self, adj_matrix, node_features):
        h = self.layer1(adj_matrix, node_features)
        h = torch.relu(h)
        h = self.layer2(adj_matrix, h)
        h = torch.relu(h)
        h = self.layer3(adj_matrix, h)
        return h
# Improved GAT model
model = ImprovedGAT(in_features=node_features.size()[1], hidden_features=8, out_features=7)
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
# train the model
model.train()
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        adj_matrix, node_features, labels = data
        optimizer.zero_grad()
        outputs = model(adj_matrix, node_features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))
# model testing
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        adj_matrix, node_features, labels = data
        outputs = model(adj_matrix, node_features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy: %d %%' % (100 * correct / total))


[1] loss: 1.842
[2] loss: 1.682
[3] loss: 1.388
[4] loss: 1.058
[5] loss: 0.843
[6] loss: 0.626
[7] loss: 0.525
[8] loss: 0.421
[9] loss: 0.454
[10] loss: 0.388
[11] loss: 0.242
[12] loss: 0.229
[13] loss: 0.227
[14] loss: 0.149
[15] loss: 0.229
[16] loss: 0.148
[17] loss: 0.122
[18] loss: 0.105
[19] loss: 0.111
[20] loss: 0.101
[21] loss: 0.094
[22] loss: 0.085
[23] loss: 0.083
[24] loss: 0.075
[25] loss: 0.083
[26] loss: 0.079
[27] loss: 0.076
[28] loss: 0.071
[29] loss: 0.191
[30] loss: 0.081
[31] loss: 0.065
[32] loss: 0.102
[33] loss: 0.069
[34] loss: 0.072
[35] loss: 0.070
[36] loss: 0.068
[37] loss: 0.067
[38] loss: 0.066
[39] loss: 0.227
[40] loss: 0.281
[41] loss: 0.146
[42] loss: 0.242
[43] loss: 0.372
[44] loss: 0.169
[45] loss: 0.156
[46] loss: 0.110
[47] loss: 0.103
[48] loss: 0.101
[49] loss: 0.098
[50] loss: 0.097
[51] loss: 0.096
[52] loss: 0.096
[53] loss: 0.095
[54] loss: 0.095
[55] loss: 0.097
[56] loss: 0.093
[57] loss: 0.095
[58] loss: 0.094
[59] loss: 0.095
[60] l

In [29]:
pred = model(test_loader,node_features)

TypeError: 'DataLoader' object is not subscriptable

In [27]:
preds = pred[idx_test]
np.savetxt('submission.txt', preds, fmt='%d')

NameError: name 'pred' is not defined